In [2]:
import numpy as np
import random 
from numba import njit

Q1: The stock market
-

In [7]:
def model(state, bull_prob, bear_prob, recession_prob, state_days):

    initial_state = random.choice(list(state.keys()))
    state_days[state[initial_state]] += 1

    for i in range(10**6):
    
        if initial_state == "bull":
            probs = bull_prob
        elif initial_state == "bear":
            probs = bear_prob
        else:
            probs = recession_prob

        next_state = random.choices(list(state.keys()), weights=probs, k=1)[0]
        initial_state = next_state
        state_days[state[initial_state]] += 1


state = {"bull": 0, "bear": 1, "recession": 2}
bull_prob = [0.9, 0.075, 0.025]
bear_prob = [0.15, 0.8, 0.05]
recession_prob = [0.25, 0.25, 0.5]
state_days = np.zeros(3)

simulation = model(state, bull_prob, bear_prob, recession_prob, state_days)
print("Giorni per stato:", state_days)


Giorni per stato: [624894, 312903, 62204]


In [10]:
%timeit model(state, bull_prob, bear_prob, recession_prob, state_days)

1.33 s ± 33.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
BULL = 0
BEAR = 1
RECESSION = 2

bull_prob = np.array([0.9, 0.075, 0.025])
bear_prob = np.array([0.15, 0.8, 0.05])
recession_prob = np.array([0.25, 0.25, 0.5])

@njit
def numba_model(n_step):
    state_days = np.zeros(3)
    initial_state = np.random.randint(0,3)
    state_days[initial_state] += 1

    for i in range(n_step):
    
        if initial_state == 0:
            probs = bull_prob
        elif initial_state == 1:
            probs = bear_prob
        else:
            probs = recession_prob

        r = np.random.random()
        if r < probs[0]:
            next_state = 0
        elif r < probs[0] + probs[1]:
            next_state = 1
        else:
            next_state = 2
            
        initial_state = next_state
        state_days[initial_state] += 1
        
    return state_days
        
simulation = numba_model(10**6)
print("Giorni per stato:", simulation)

Giorni per stato: [625790. 311855.  62356.]


In [21]:
%timeit model(state, bull_prob, bear_prob, recession_prob, state_days)
%timeit numba_model(10**6)

2.33 s ± 82.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.98 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


-----------
Q2: Consistent plotting
-

In [8]:
def plot_decorator(func):
    def wrapper(*args, **kwargs):
        import matplotlib.pyplot as plt
        from matplotlib.backends.backend_pdf import PdfPages

        plt.rcParams.update({
            "font.size": 14,
            "axes.labelsize": 14,
            "axes.titlesize": 16,
            "xtick.labelsize": 12,
            "ytick.labelsize": 12,
            "figure.figsize": (8, 5),
            "figure.dpi": 120,
            "savefig.bbox": "tight",
        })

        pdf_name = func.__name__ + ".pdf"
        pp = PdfPages(pdf_name)

        fig = plt.figure()
        result_fig = func(*args, **kwargs)

        if not isinstance(result_fig, plt.Figure):
            raise TypeError(
                f"Function {func.__name__} must return a matplotlib Figure."
            )

        result_fig.savefig(pp, format='pdf', bbox_inches='tight')
        pp.close()

        return result_fig

    return wrapper
